# ЗАГРУЗКА ПОДГОТОВЛЕННОГО ДАТАФРЕЙМА В БД SQL
# код для вставки в блокнот/программу

## imports

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# импортируем библиотеки
import glob
import os
import socket
import sys
import pandas as pd
from datetime import datetime as dt_


In [2]:
time_0 = dt_.now()

## constants

In [ ]:

table_schema = 'recipes'


### ❕ Получение датафрейма (вставь свой код сюда)

In [ ]:

# маска имени файла с выгрузкой
fn_mask = r'..\uploadings\DAT4853_week_from_20??-??-??_to_20??-??-??_long.parquet'


In [ ]:

# получение списка всех файлов с выгрузками
ls_fn_full = glob.glob(fn_mask)


In [ ]:

# инициализируем пустой датафрейм, из которого данные будут мигрировать в SQL
df = pd.DataFrame()
# итерируемся по списку файлов
for fn in ls_fn_full:
    # читаем очередной файл во временный датафрейм
    df_temp = pd.read_parquet(fn)
    # добавляем данные в конец результирующего фрейма
    df = pd.concat([df, df_temp], ignore_index=True)


## read dictionaries in dfs

In [12]:
# функция, которая проверяет отсутствие имен файлов из кортежа tup_fns в содержимом папки path_
def check_missing_files(tup_fns, path_):
    # Получаем список файлов в указанной папке
    files_in_directory = os.listdir(path_)
    
    # Создаём список файлов, которых нет в папке
    missing_files = [file for file in tup_fns if file not in files_in_directory]
    
    # Проверка и вывод результата
    if missing_files:
        print(f"Следующие файлы отсутствуют в папке '{path_}': {', '.join(missing_files)}")
        sys.exit()
    else:
        print(f"Все необходимые для работы программы служебные файлы присутствуют в папке '{path_}'.")
    
    return missing_files

# кортеж с именами необходимых для работы программы служебных файлов
tup_fns = (
    'user_functions.py',
    'paths_and_constants.py',
    'dict_sql_queries.py',
    'get_dicts_v_06.py',
)


# получаем имя компьютера
hostname = socket.gethostname()

# если программа выполняется на сервере
if hostname == '304-007':
    # указываем путь к общим документам
    path_ = r'C:\Users\Public\dictionaries'
# иначе (если программа выполняется на своём рабочем ПК)
else:
    # указываем путь к документам активного пользователя
    path_ = os.path.expanduser('~\\Documents\\')

# проверяем отсутствие имен файлов из кортежа tup_fns в содержимом папки path_
check_missing_files(tup_fns, path_)

# добавляем путь к системному пути поиска модулей (путь к папке, в которой Python ищет модули)
sys.path.append(path_)

# далее импорт пользовательских библиотек идёт с пути `path_`
try:
    import get_dicts_v_06
    from user_functions import df_2_sql
except:
    print(f'\nДоп. модули, необходиимые для работы кода, НЕ НАЙДЕНЫ по адресу "{path_}"')

# использование пользовательской функции для получения словарей из SQL
# можно ввести номера словарей через пробел в скобки, если словари заранее известны
dict_dicts, tables = get_dicts_v_06.main('1 2')  #  <------------------------------------------------------------------------------------ ВВОД!

# Цикл по индексам от 0 до длины списка tables
for i in range(len(tables)):
    try:
        # Используем exec() для динамического создания переменной с именем таблицы (tables[i][0]) 
        # и присваивания ей первого элемента (таблицы) из соответствующего словаря dict_dicts
        exec(f"{tables[i][0]} = dict_dicts[tables[i][0]][0]")
        # Вывод информации о таблице, включая её название, заголовок, и дату загрузки в БД
        print(f"\nТаблицей '{tables[i][0]}' можно пользоваться. "
              f"Её название: {dict_dicts[tables[i][0]][1]} "
              f"Она была загружена в БД {dict_dicts[tables[i][0]][2]}")
        # Отображаем первые две строки таблицы с помощью функции display()
        display(dict_dicts[tables[i][0]][0].head(2))
    except:
        # print(f"\nСкорее всего, таблицы '{tables[i][0]}' ещё не существует в БД")
        # Если происходит ошибка (например, если таблицы ещё нет в БД), просто пропускаем и ничего не выводим.
        pass



    ```
| table          | Маска имени файла                     | Клавиша |
| -------------- | ------------------------------------- | ------- |
| all            | <Все словари>                         |    0    |
| df_names       | Словарь_Названия_20YY_MM_DD.xlsx      |    1    |
| df_mo          | Словарь_МО_20YY_MM_DD.xlsx            |    2    |
| df_mkb_10      | Словарь_МКБ_10_20YY_MM_DD.xlsx        |    3    |
| df_orders      | Словарь_Приказы_20YY_MM_DD.xlsx       |    4    |
| df_spec        | Словарь_Специальности_20YY_MM_DD.xlsx |    5    |
| df_app         | Словарь_Назначения_20YY_MM_DD.xlsx    |    6    |
| df_profile_vop | Словарь_Профиль_ВОП_20YY_MM_DD.xlsx   |    7    |
```

Вы выбрали: ['df_names', 'df_mo', 'df_mkb_10', 'df_orders', 'df_spec', 'df_app', 'df_profile_vop']

Не удалось загрузить словарь "df_app" из-за Table df_app not found

Словари записаны в словарь словарей `dict_dicts`


In [6]:
df_for_dict_rename = df_names[['rus_name', 'eng_name']]
df_for_dict_rename = df_for_dict_rename.set_index('rus_name')['eng_name'].to_dict()
# display(df_for_dict_rename)


In [7]:
user_s_answer = input('Нажмите "Перейти" и при необходимости поменяйте "вводные" в ячейке после исполняемой, после чего введите любой символ в это поле: ')

## Аргументы ⌨

In [8]:
# имя таблицы в БД
table_name = '5213_bva'
# составной первичный ключ, либо список из одного столбца / пустой список
ls_pk = ['emias_id', 'mkb_10']
# столбцы для индекса таблицы
ls_index = ['oms', 'mo', ]
# сколько строк сгружать за итерацию
rows = (500_000, 8_900_000)[0]

### ❌ delete one cell below: создание песочных данных

In [9]:
if df.duplicated(subset=['emias_id', 'mkb_10']).sum() > 0:
    df = df.drop_duplicates(subset=['emias_id', 'mkb_10']).reset_index(drop=True)

df = df.loc[0:499_999, :]

## create table in postgresql db

In [10]:
df.rename(columns=df_for_dict_rename, inplace=True)

df_2_sql(df, table_schema, table_name, ls_pk, ls_index, rows=rows)

In [ ]:
# проверяем правильность нейминга столбцов
df.info()

In [12]:
print(f'done in {((dt_.now() - time_0).seconds / 60):.1f} minutes')

done in 0.7 minutes
